# A/B Testing Project

## Contents  <a name='back'></a>

1. [Introduction](#introduction)
2. [Part 1: Hypotheses priorization](#hypotheses)

## Introduction <a name='introduction'></a>

This proyect is for the A/B Testing sprint on Tripleten's Data Analyst course. It consists on two parts:

- Part 1 - Prioritizing hypotheses: We have a list of hypotheses that we need to prioritize according to ICE and RICE, and draw conclusions.
- Part 2 - A/B Test Analysis: For this part we have data on visitors and orders. Our objective is to determine the results of the test.

For this project we'll be using the following:
- Python 3.9.5
- Pandas 1.2.4
- Matplotlib.pyplot 3.3.4
- Numpy 1.20.3
- Scipy.stats 1.7.3

Versions were chosen so they match as closely as possible the versions available on the Tripleten servers

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import datetime as dt